In [ ]:
import sys
sys.path.insert(1, '/mnt/d/PowerTAC/Python/python_utils/helper')

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt
from read_mongo_collection import HelperToReadMongo

# Adding background to the graph (few good backgrounds below)
# plt.style.use("fivethirtyeight")
plt.style.use("ggplot")
# plt.style.use("seaborn-*")

# See complete data
np.set_printoptions(threshold=np.inf)

In [ ]:
helper_mongo = HelperToReadMongo()

In [ ]:
# Specify names of all the databases here
list_of_databases = ['PowerTAC2020_Wholesale_Test_VV21', 'PowerTAC2020_Wholesale_Test_VV18']

In [ ]:
# View the Collections in all the databases

def view_collections(list_of_databases):
    
    for database in list_of_databases:
        
        db = helper_mongo.connect_to_mongo(database)
        print(database + "\n" + "-"*35)
        print(db.list_collection_names())
        print("\n")

# Function Calling
view_collections(list_of_databases)

In [ ]:
vv21_submitted_bids =  helper_mongo.query_to_mongo('PowerTAC2020_Wholesale_Test_VV21', 'Submitted_Bid_Information')
vv18_submitted_bids =  helper_mongo.query_to_mongo('PowerTAC2020_Wholesale_Test_VV18', 'Submitted_Bid_Information')

# consider only bids
vv21_submitted_bids = vv21_submitted_bids[vv21_submitted_bids['LimitPrice'] < 0.0]
vv18_submitted_bids = vv18_submitted_bids[vv18_submitted_bids['LimitPrice'] < 0.0]

In [ ]:
vv21_market_clearing =  helper_mongo.query_to_mongo('PowerTAC2020_Wholesale_Test_VV21', 'MarketTransaction_Info')
vv18_market_clearing =  helper_mongo.query_to_mongo('PowerTAC2020_Wholesale_Test_VV18', 'MarketTransaction_Info')

# consider only bids
vv21_market_clearing = vv21_market_clearing[vv21_market_clearing['MCP'] < 0.0]
vv18_market_clearing = vv18_market_clearing[vv18_market_clearing['MCP'] < 0.0]

In [ ]:
vv21_cleared_trade =  helper_mongo.query_to_mongo('PowerTAC2020_Wholesale_Test_VV21', 'ClearedTrade_Info')
vv18_cleared_trade =  helper_mongo.query_to_mongo('PowerTAC2020_Wholesale_Test_VV18', 'ClearedTrade_Info')

In [ ]:
vv21_submitted_bids['Proximity'] = vv21_submitted_bids['Execution_Timeslot'] - vv21_submitted_bids['Bidding_Timeslot']
vv18_submitted_bids['Proximity'] = vv18_submitted_bids['Execution_Timeslot'] - vv18_submitted_bids['Bidding_Timeslot']

vv21_market_clearing['Proximity'] = vv21_market_clearing['Execution_Timeslot'] - vv21_market_clearing['Bidding_Timeslot']
vv18_market_clearing['Proximity'] = vv18_market_clearing['Execution_Timeslot'] - vv18_market_clearing['Bidding_Timeslot']

vv21_cleared_trade['Proximity'] = vv21_cleared_trade['Execution_Timeslot'] - vv21_cleared_trade['Bidding_Timeslot']
vv18_cleared_trade['Proximity'] = vv18_cleared_trade['Execution_Timeslot'] - vv18_cleared_trade['Bidding_Timeslot']

In [ ]:
vv21_lp = abs(vv21_submitted_bids.groupby('Proximity').mean()['LimitPrice'])
vv18_lp = abs(vv18_submitted_bids.groupby('Proximity').mean()['LimitPrice'])

vv21_cp = abs(vv21_market_clearing.groupby('Proximity').mean()['MCP'])
vv18_cp = abs(vv18_market_clearing.groupby('Proximity').mean()['MCP'])

vv21_mcp = vv21_cleared_trade.groupby('Proximity').mean()['MCP']
vv18_mcp = vv18_cleared_trade.groupby('Proximity').mean()['MCP']

In [ ]:
vv21_lq = abs(vv21_submitted_bids.groupby('Proximity').mean()["Broker's_Bidded_Quantity"])
vv18_lq = abs(vv18_submitted_bids.groupby('Proximity').mean()["Broker's_Bidded_Quantity"])

vv21_cq = abs(vv21_market_clearing.groupby('Proximity').mean()["Broker's_Cleared_Quantity"])
vv18_cq = abs(vv18_market_clearing.groupby('Proximity').mean()["Broker's_Cleared_Quantity"])

vv21_mcq = vv21_cleared_trade.groupby('Proximity').mean()["Net_Cleared_Quantity"]
vv18_mcq = vv18_cleared_trade.groupby('Proximity').mean()["Net_Cleared_Quantity"]

In [ ]:
 # plotting
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18,8))
fig.tight_layout(pad=5)

axes[0].set_title("Limitprice Comparison")
axes[0].plot(vv21_lp, label="VV21")
axes[0].plot(vv18_lp, label="VV18")
axes[0].set_xlabel("Proximity")
axes[0].set_ylabel("Limitprice")
axes[0].legend()

axes[1].set_title("Clearing-price Comparison")
axes[1].plot(vv21_cp, label="VV21")
axes[1].plot(vv18_cp, label="VV18")
axes[1].set_xlabel("Proximity")
axes[1].set_ylabel("MCP")
axes[1].legend()

axes[2].set_title("MCP Comparison")
axes[2].plot(vv21_mcp, label="VV21")
axes[2].plot(vv18_mcp, label="VV18")
axes[2].set_xlabel("Proximity")
axes[2].set_ylabel("MCP")
axes[2].legend()

plt.show()

In [ ]:
 # plotting
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18,8))
fig.tight_layout(pad=5)

axes[0].set_title("Limitquantity Comparison")
axes[0].plot(vv21_lq, label="VV21")
axes[0].plot(vv18_lq, label="VV18")
axes[0].set_xlabel("Proximity")
axes[0].set_ylabel("Broker's bid Quantity")
axes[0].legend()

axes[1].set_title("Clearing-quantity Comparison")
axes[1].plot(vv21_cq, label="VV21")
axes[1].plot(vv18_cq, label="VV18")
axes[1].set_xlabel("Proximity")
axes[1].set_ylabel("Broker's Cleared Quantity")
axes[1].legend()

axes[2].set_title("MCQ Comparison")
axes[2].plot(vv21_mcq[:23], label="VV21")
axes[2].plot(vv18_mcq[:23], label="VV18")
axes[2].set_xlabel("Proximity")
axes[2].set_ylabel("Net Cleared Quantity")
axes[2].legend()

plt.show()

In [ ]:
 # plotting
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18,8))
fig.tight_layout(pad=5)

axes[0].set_title("Perfomance Comparison")
axes[0].plot(vv21_mcp, label="MCP")
axes[0].set_xlabel("Proximity")
axes[0].legend()

axes[1].set_title("Perfomance Comparison")
axes[1].plot(vv21_lq, label="VV21 CQ")
axes[1].plot(vv18_lq, label="VV18 CQ")
axes[1].set_xlabel("Proximity")
axes[1].legend()

plt.show()